https://cs.brown.edu/courses/cs195u/lectures/04_advancedCollisionsAndPhysics.pdf

In [ ]:
import numpy as np
import jax 
import jax.numpy as jnp 
from typing import List

In [ ]:
def support_idx(pts, direction):
    '''
        Find the index of the support point of a polygon's pts that is has the largest dot product with direction
        Inputs:
            pts: (N, D) array of points
            direction: (D,) array
    '''
    assert pts.shape[1] == direction.shape[0], "Dimension mismatch"
    return jnp.argmax(jnp.dot(pts, direction))

def support(pts, direction):
    '''
        Find the support point of a polygon's pts that is has the largest dot product with direction
        Inputs:
            pts: (N, D) array of points
            direction: (D,) array
    '''
    return pts[support_idx(pts, direction)]

def support2(pts1, pts2, direction):
    '''
        Find the support point of the Minkowski difference pts1 - pts2 w.r.t. direction 
        Inputs:
            pts1: (N, D) array of points
            pts2: (M, D) array of points
            direction: (D,) array
        Returns:
            support_point: (D,) array
    '''
    assert pts1.shape[1] == direction.shape[0], "Dimension mismatch"
    assert pts2.shape[1] == direction.shape[0], "Dimension mismatch"
    return support(pts1, direction) - support(pts2, -direction)

In [ ]:
def do_simplex(simplex_list: List[np.ndarray], tol=1e-3):
    '''
        Check if the simplex contains the origin
        Inputs:
            simplex_list: (N, 3) array of simplex points
            direction: direction
        Returns:
            True if the simplex contains the origin, False otherwise
    '''
    N = len(simplex_list)
    D = simplex_list[0].shape[0]
    
    assert D == 3, "Simplex must be 3D"
    
    # check if the last simplex point is the origin
    if np.linalg.norm(simplex_list[-1]) < tol:
            return True, None, None
    
    if N == 1: # 0-simplex a point
        # if the point not origin, return the direction to the origin
        return False, simplex_list, -simplex_list[-1]
    elif N == 2: # 1-simplex a line
        return do_one_simplex(simplex_list)
    elif N == 3: # 2-simplex a triangle
        return do_two_simplex(simplex_list)
    
def do_one_simplex(simplex_list):
    '''
        Check if the simplex contains the origin
        Inputs:
            simplex: (2,3) array of simplex points
            tol: tolerance
        Returns:
            The direction to the origin and the updated simplex list
    '''
    N, D = simplex_list.shape

    assert N == 2, "Simplex must be one-simplex"
    assert D == 3, "Simplex must be 3D"

    # Get the two points of the line
    A = simplex_list[0]
    B = simplex_list[1]
    
    # Compute the direction vector of the line
    AB = B - A
    
    # Compute the direction vector from the line to the origin
    AO = -A
    
    # Compute the direction vector normal to the line
    direction = np.cross(np.cross(AB, AO), AB)
    
    return False, simplex_list, direction


def do_two_simplex(simplex_list):
    '''
        Check if the simplex contains the origin
        Inputs:
            simplex: (3,3) array of simplex points
            tol: tolerance
        Returns:
            The direction to the origin 
    '''
    N, D = simplex_list.shape

    assert N == 3, "Simplex must be two-simplex"
    assert D == 3, "Simplex must be 3D"

    # Get the three points of the triangle
    A = simplex_list[2]
    B = simplex_list[1]
    C = simplex_list[0]
    
    # Compute the direction vectors of the triangle
    AB = B - A
    AC = C - A
    AO = -A
    
    # compute the normal vector of the triangle
    normal = np.cross(AB, AC)
    
    if np.dot(np.cross(normal, AC), AO) > 0:
        # The origin is on the side of CA or A region
        if np.dot(AC, AO) > 0:
            # The origin is on the side of CA
            new_simplex_list = [C, A]
            direction = np.cross(np.cross(AC, AO), AC)
            return False, new_simplex_list, direction
        else:
            new_simplex_list = [A]
            direction = AO
            return False, new_simplex_list, direction
    else:
        # The origin is on the side of AB or B region
        if np.dot(np.cross(AB, normal), AO) > 0:
            # The origin is on the side of AB
            if np.dot(AB, AO) > 0:
                new_simplex_list = [B, A]
                direction = np.cross(np.cross(AB, AO), AB)
                return False, new_simplex_list, direction
            else:
                new_simplex_list = [A]
                direction = AO
                return False, new_simplex_list, direction
        else:
            # The origin is inside the triangle
            return True, None, None
            


In [ ]:
def gjk(pts1, pts2, tol=1e-3):
    '''
        Gilbert-Johnson-Keerthi algorithm for computing the distance between two convex sets
        Inputs:
            pts1: (N, 2) array of points
            pts2: (M, 2) array of points
        Outputs:
            distance: float, distance between the two sets
            closest_pts: (2, D) array of points, closest points on each set
    '''
    assert pts1.shape[1] == pts2.shape[1], "Dimension mismatch"
    assert pts1.shape[0] > 0, "Empty set"
    assert pts2.shape[0] > 0, "Empty set"
    assert pts1.shape[1] == 2, "only 2D supported"
    assert pts2.shape[1] == 2, "only 2D supported"


    # pad the points to 3D
    pts1 = np.hstack([pts1, np.zeros((pts1.shape[0], 1))])
    pts2 = np.hstack([pts2, np.zeros((pts2.shape[0], 1))])
    
    direction = np.array([1, 1, 0])
    
    # Initialize simplex
    simplex = [support2(pts1, pts2, direction)]

    # Initialize direction
    direction = -simplex[-1]

    # Iterate until convergence
    while True:
        # Get a new simplex in the direction opposite to the previous simplex
        A = support2(pts1, pts2, direction)
        
        # Check if we need to pass origin is in simplex
        # The Minkowski difference is convex, so if we did not pass the origin 
        # when we walked from vertex S to vertex A, then we know that there is no collision         
        if np.dot(direction, A) <= 0:
            return False, []
        
        simplex.append(A)
        

# Hard Code for box

In [16]:
from typing import Tuple

In [58]:
def minkowski_sum_of_box_and_box_points(box1_points: np.ndarray,
                                        box2_points: np.ndarray) -> np.ndarray:
  """Batched Minkowski sum of two boxes (counter-clockwise corners in xy).

  The last dimensions of the input and return store the x and y coordinates of
  the points. Both box1_points and box2_points needs to be stored in
  counter-clockwise order. Otherwise the function will return incorrect results
  silently.

  Args:
    box1_points: Tensor of vertices for box 1, with shape:
      (num_boxes, num_points_per_box, 2).
    box2_points: Tensor of vertices for box 2, with shape:
      (num_boxes, num_points_per_box, 2).

  Returns:
    The Minkowski sum of the two boxes, of size (num_boxes,
    num_points_per_box * 2, 2). The points will be stored in counter-clockwise
    order.
  """
  NUM_BOX_1 = box1_points.shape[0]
  NUM_BOX_2 = box2_points.shape[0]
  NUM_VERTICES_IN_BOX = box1_points.shape[1]
  assert NUM_VERTICES_IN_BOX == 4, "Only support boxes"
  # Hard coded order to pick points from the two boxes. This is a simplification
  # of the generic convex polygons case. For boxes, the adjacent edges are
  # always 90 degrees apart from each other, so the index of vertices can be
  # hard coded.
  point_order_1 = [0, 0, 1, 1, 2, 2, 3, 3]
  point_order_2 = [0, 1, 1, 2, 2, 3, 3, 0]

  box1_start_idx, downmost_box1_edge_direction = _get_downmost_edge_in_box(
      box1_points)
  box2_start_idx, downmost_box2_edge_direction = _get_downmost_edge_in_box(
      box2_points)

  # The cross-product of the unit vectors indicates whether the downmost edge
  # in box2 is pointing to the left side (the inward side of the resulting
  # Minkowski sum) of the downmost edge in box1. If this is the case, pick
  # points from box1 in the order `point_order_2`, and pick points from box2 in
  # the order of `point_order_1`. Otherwise, we switch the order to pick points
  # from the two boxes, pick points from box1 in the order of `point_order_1`,
  # and pick points from box2 in the order of `point_order_2`.
  # Shape: (num_boxes, 1)
  condition = (
      np.cross(downmost_box1_edge_direction, downmost_box2_edge_direction)
      >= 0.0
  )
  # box1_point_order of size [num_boxes, num_points_per_box * 2 = 8, 1].
  box1_point_order = np.where(condition, point_order_2, point_order_1)[..., None]

  # Shift box1_point_order by box1_start_idx, so that the first index in
  # box1_point_order is the downmost vertex in the box.
  box1_point_order = np.mod(box1_point_order + box1_start_idx,
                                 NUM_VERTICES_IN_BOX)
  # Gather points from box1 in order.
  # ordered_box1_points is of size [num_boxes, num_points_per_box * 2, 2].
  ordered_box1_points = np.take_along_axis(box1_points, box1_point_order, axis=-2)

  # Gather points from box2 as well.
  box2_point_order = np.where(condition, point_order_1, point_order_2)[..., None]
  box2_point_order = np.mod(box2_point_order + box2_start_idx,
                                 NUM_VERTICES_IN_BOX)
  ordered_box2_points = np.take_along_axis(box2_points, box2_point_order, axis=-2)
  # ordered_box2_points = box2_points[np.arange(NUM_BOX_2), box2_point_order, :]
  minkowski_sum = ordered_box1_points + ordered_box2_points
  return minkowski_sum

def _get_downmost_edge_in_box(box: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Finds the downmost (lowest y-coordinate) edge in the box.

    Note: We assume box edges are given in a counter-clockwise order, so that
    the edge which starts with the downmost vertex (i.e. the downmost edge) is
    uniquely identified.

    Args:
    box: (num_boxes, num_points_per_box, 2). The last dimension contains the x-y
        coordinates of corners in boxes.

    Returns:
    A tuple of two tensors:
        downmost_vertex_idx: The index of the downmost vertex, which is also the
        index of the downmost edge. Shape: (num_boxes, 1, 1).
        downmost_edge_direction: The tangent unit vector of the downmost edge,
        pointing in the counter-clockwise direction of the box.
        Shape: (num_boxes, 1, 2).
    """
    # The downmost vertex is the lowest in the y dimension.
    # Shape: (num_boxes, 1).
    
    NUM_BOX, NUM_VERTICES_IN_BOX, _ = box.shape
    assert NUM_VERTICES_IN_BOX == 4, "Only support boxes"
    downmost_vertex_idx = np.argmin(box[..., 1], axis=-1)[..., None, None]

    # Find the counter-clockwise point edge from the downmost vertex.
    edge_start_vertex = np.take_along_axis(box, downmost_vertex_idx, axis=1)
    # edge_start_vertex = box[np.arange(NUM_BOX), downmost_vertex_idx, :]
    edge_end_idx = np.mod(downmost_vertex_idx + 1, NUM_VERTICES_IN_BOX)
    edge_end_vertex = np.take_along_axis(box, edge_end_idx, axis=1)
    # edge_end_vertex = box[np.arange(NUM_BOX), edge_end_idx, :]

    # Compute the direction of this downmost edge.
    downmost_edge = edge_end_vertex - edge_start_vertex
    downmost_edge_length = np.linalg.norm(downmost_edge, axis=-1, keepdims=True)
    downmost_edge_direction = downmost_edge / downmost_edge_length
    return downmost_vertex_idx, downmost_edge_direction

In [62]:
def _get_edge_info(
    polygon_points: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Computes properties about the edges of a polygon.

  Args:
    polygon_points: Tensor containing the vertices of each polygon, with
      shape (num_polygons, num_points_per_polygon, 2). Each polygon is assumed
      to have an equal number of vertices.

  Returns:
    tangent_unit_vectors: A unit vector in (x,y) with the same direction as
      the tangent to the edge. Shape: (num_polygons, num_points_per_polygon, 2).
    normal_unit_vectors: A unit vector in (x,y) with the same direction as
      the normal to the edge.
      Shape: (num_polygons, num_points_per_polygon, 2).
    edge_lengths: Lengths of the edges.
      Shape (num_polygons, num_points_per_polygon).
  """
  # Shift the polygon points by 1 position to get the edges.
  # Shape: (num_polygons, 1, 2).
  first_point_in_polygon = polygon_points[:, 0:1, :]
  # Shape: (num_polygons, num_points_per_polygon, 2).
  shifted_polygon_points = np.concatenate(
      [polygon_points[:, 1:, :], first_point_in_polygon], axis=-2)
  # Shape: (num_polygons, num_points_per_polygon, 2).
  edge_vectors = shifted_polygon_points - polygon_points

  # Shape: (num_polygons, num_points_per_polygon).
  edge_lengths = np.linalg.norm(edge_vectors, axis=-1)
  # Shape: (num_polygons, num_points_per_polygon, 2).
  tangent_unit_vectors = edge_vectors / edge_lengths[:, :, None]
  # Shape: (num_polygons, num_points_per_polygon, 2).
  normal_unit_vectors = np.stack(
      [-tangent_unit_vectors[..., 1], tangent_unit_vectors[..., 0]], axis=-1)
  return tangent_unit_vectors, normal_unit_vectors, edge_lengths



In [68]:
def signed_distance_from_point_to_convex_polygon(
    query_points: np.ndarray, polygon_points: np.ndarray) -> np.ndarray:
  """Finds the signed distances from query points to convex polygons.

  Each polygon is represented by a 2d tensor storing the coordinates of its
  vertices. The vertices must be ordered in counter-clockwise order. An
  arbitrary number of pairs (point, polygon) can be batched on the 1st
  dimension.

  Note: Each polygon is associated to a single query point.

  Args:
    query_points: (batch_size, 2). The last dimension is the x and y
      coordinates of points.
    polygon_points: (batch_size, num_points_per_polygon, 2). The last
      dimension is the x and y coordinates of vertices.

  Returns:
    A tensor containing the signed distances of the query points to the
    polygons. Shape: (batch_size,).
  """
  tangent_unit_vectors, normal_unit_vectors, edge_lengths = _get_edge_info(polygon_points)

  # Expand the shape of `query_points` to (num_polygons, 1, 2), so that
  # it matches the dimension of `polygons_points` for broadcasting.
  query_points = query_points[:, None, :]
  
  # Compute query points to polygon points distances.
  # Shape (num_polygons, num_points_per_polygon, 2).
  vertices_to_query_vectors = query_points - polygon_points
  
  # Shape (num_polygons, num_points_per_polygon).
  vertices_distances = np.linalg.norm(vertices_to_query_vectors, axis=-1)

  # Query point to edge distances are measured as the perpendicular distance
  # of the point from the edge. If the projection of this point on to the edge
  # falls outside the edge itself, this distance is not considered (as there)
  # will be a lower distance with the vertices of this specific edge.

  # Make distances negative if the query point is in the inward side of the
  # edge. Shape: (num_polygons, num_points_per_polygon).
  edge_signed_perp_distances = np.sum(
      -normal_unit_vectors * vertices_to_query_vectors, axis=-1)

  # If `edge_signed_perp_distances` are all less than 0 for a
  # polygon-query_point pair, then the query point is inside the convex polygon.
  is_inside = np.all(edge_signed_perp_distances <= 0, axis=-1)

  # Project the distances over the tangents of the edge, and verify where the
  # projections fall on the edge.
  # Shape: (num_polygons, num_edges_per_polygon).
  projection_along_tangent = np.sum(
      tangent_unit_vectors * vertices_to_query_vectors, axis=-1)
  projection_along_tangent_proportion = projection_along_tangent/edge_lengths
  
  # Shape: (num_polygons, num_edges_per_polygon).
  is_projection_on_edge = np.logical_and(
      projection_along_tangent_proportion >= 0.0,
      projection_along_tangent_proportion <= 1.0)

  # If the point projection doesn't lay on the edge, set the distance to inf.
  edge_perp_distances = np.abs(edge_signed_perp_distances)
  edge_distances = np.where(is_projection_on_edge,
                            edge_perp_distances, np.inf)

  # Aggregate vertex and edge distances.
  # Shape: (num_polyons, 2 * num_edges_per_polygon).
  edge_and_vertex_distance = np.concatenate([edge_distances, vertices_distances],
                                       axis=-1)
  # Aggregate distances per polygon and change the sign if the point lays inside
  # the polygon. Shape: (num_polygons,).
  min_distance = np.min(edge_and_vertex_distance, axis=-1)
  signed_distances = np.where(is_inside, -min_distance, min_distance)
  return signed_distances

In [2]:
# autoreload 
%load_ext autoreload
%autoreload 2

In [72]:
# two boxes with corners in counter-clockwise order
import numpy as np
box1 =np.array([[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]]])
box2 = np.array([[[1, 1.5], [1.5, 1.5], [1.5, 2.5], [1, 2.5]]])
min_diff = minkowski_sum_of_box_and_box_points(box1, -box2)

signed_distance_from_point_to_convex_polygon(np.array([[0,0]]), min_diff)


array([0.5])

In [55]:
from waymo_open_dataset.utils.geometry_utils import minkowski_sum_of_box_and_box_points as min_tf
import tensorflow as tf
box1_tf = tf.convert_to_tensor(box1, dtype=tf.float32)
# box2_tf = tf.convert_to_tensor(box2, dtype=tf.float32)
min_tf(box1_tf, -box1_tf)

<tf.Tensor: shape=(2, 8, 2), dtype=float32, numpy=
array([[[-1. , -1. ],
        [ 0. , -1. ],
        [ 1. , -1. ],
        [ 1. ,  0. ],
        [ 1. ,  1. ],
        [ 0. ,  1. ],
        [-1. ,  1. ],
        [-1. ,  0. ]],

       [[ 0. , -2. ],
        [ 0.5, -1.5],
        [ 1. ,  0. ],
        [ 0.5,  1.5],
        [ 0. ,  2. ],
        [-0.5,  0.5],
        [-1. ,  0. ],
        [-0.5, -0.5]]], dtype=float32)>

In [38]:
idx = np.array([0,1])[:, None, None]
np.take_along_axis(box1, idx, axis=1)

array([[[0. , 0. ]],

       [[1.5, 0.5]]])